## 1. 함수의 선언
#### 1) 내장함수 (built-in)
* web3.eth.getAccounts()
* keccak256()
* sha256()

#### 2) 사용자 정의 함수 (user-defined)
```
function (param types) {internal|external} [pure|view|payable] [returns (return types)] varName;
```

## 2. 생성자
* constructor()
 * 자바와 같이 굳이 구현하지 않아도 됨
 * 하지만 overloading 불가능
 * return 값 없음
 
## 3. 수식어
* solidity에서 함수를 실행하면, 그 결과로 인해 블록체인의 변화 유무에 따라 구분한다.
 #### view
 * 블록체인의 상태를 변경하지 않는 함수
 * 읽기 전용(read only)
 
 #### pure
 * 블록체인의 상태를 변경하지 않는 함수
 * 읽지도 않는 경우
 
이 외에는 transaction이 발생하므로 transaction hash가 반환된다.

view와 pure의 경우는 결과 값을 확인 하는 방법은 event를 사용해서만 가능하다.

#### view 불가능
 * 상태변수 state variables를 읽는 경우,
 * 잔고 this.balance 또는 address.balance를 읽는 경우,
 * 전역변수 block, tx, msg (msg.sig, msg.data 제외)를 읽는 경우,
 * pure로 표기되지 않은 함수를 호출하는 경우,
 * opcodes로 작성된 인라인 코드
 
#### pure 불가능
 * 상태변수를 저장하는 경우 (당연히 블록체인에 쓰는 경우이다)
 * 이벤트 발생 (이벤트가 발생하면 로그에 기록이 남겨진다.)
 * 다른 컨트랙 생성
 * selfdestruct 컨트랙을 삭제하는 경우
 * call 함수로 송금하는 경우
 * view 또는 pure 아닌 함수를 호출하는 경우
 * low-level calls을 호출하는 경우
 * opcodes로 작성된 인라인 코드

## 함수통해 살펴보기

In [65]:
%%writefile src/FuctionTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract FunctionTest {
    int x;
    //constructor
    constructor() public {
        x = 0;
    }
    // updating state var x
    function incrementX() public {
        x += 1;
    }
    // call when x = 0
    function doubleX() public {
        X2();
    }
    // float not supported. try 0, 1/3...
    function divideBy(int by) view public returns(int) {
        return x/by;
    }
    // actually NONE returned
    function getX_() view public returns(int) {
        return x;        
    }
    // try pure
    function getX() view public returns(int) {
        return x;
    }
    function getBalance() view public returns(uint) {
        return(address(this).balance);
    }
    // 'payable' means that msg.value can be deposited
    function deposit() public payable {
    }
    // can not be used in public
    function X2() internal {
        x *= 2;
    }
    function getBlockNumber() view public returns(uint) {
        return block.number;
    }
}

Overwriting src/FuctionTest.sol


In [66]:
!solc src/FuctionTest.sol

Compiler run successful, no output requested.


* constructor()	생성자는 앞에 위치시킨다.
* increatementX()	state var를 수정하므로 view, pure가 아님.
* doubleX()	내부함수 internal X2()를 호출함. state var 수정하므로 view, pure 아님.
* divideBy(int by)	0으로 나누어도 오류를 발생하지 않는다. 아직 소수는 지원하지 않으므로 필요한 경우 라이브러리를 사용한다.
* getX_()	view, pure가 아니므로returns(int)해도 반환 값을 받지 못함. 해시값을 돌려줌.
* getX()	state var를 읽으므로 view를 사용한다. pure를 사용하지 않는다.
* getBalance()	잔고 조회는 view도 가능함.
* deposit()	payable로 선언해야 입금이 가능. 함수가 비워있어도 msg.value를 입금.
* X2()	internal로 선언되어서 내부에서만 사용가능.
* getBlockNumber()	전역변수를 읽을 경우 view

# -------------------------------------------------------------------------

## modifier
##### 함수의 제약조건을 지정
* modifier를 적용하면 if문을 제거하는 효과

# 실습문제: 밑줄 명령어 적용
modifier를 사용할 때 밑줄 underscore _ 명령어를 통해 어디에 호출 소스코드가 포함되는지 분명히 할 수 있다. 아래 코드에서 포함되는 시점을 전 또는 후로 정할 수 있다.

In [67]:
%%writefile src/UnderscoreTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract UnderscoreTest {
    string season = "";
    function getSeason() view public returns(string memory) {
        return season;
    }
    function setWinter() public setSummerAfter {
        season = "winter";
    }
    function setSpring() public setSummerBefore {
        season = "spring";
    }
    modifier setSummerAfter() {
        season = "summer";
        _;
    }
    modifier setSummerBefore() {
        _;
        season = "summer";
    }
}

Writing src/UnderscoreTest.sol


* 10: setSummerAfter(이게 modifier) .. 아래 15번째를 재약조건으로 하고 출력
* 출력은 윈터가 된다 왜?? _; 이 문법 때문이다.
* 즉 17: season = summer은 _에 들어가고 11의 winter가 나온다.
* 정리하면 10 하면 17의 summer지만 _에 11의 winter가 들어간다.
* 13을 실행하면 20번에 _에 spring이 들어가고 아래 summer가 호출된다.

In [68]:
!solc src/UnderscoreTest.sol

Compiler run successful, no output requested.


# -------------------------------------------------------------------------

## 실습문제: 은행 BankV3
앞의 은행 코드에 fallabck, modifier를 넣어서 수정한다.

## 1단계 컨트랙 개발

In [69]:
%%writefile src/BankV3.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4; 

contract BankV3 {
    address owner;
    uint balance;
    uint256 timeToDeposit;
    
    event PrintLog(string);
    event Sent(address from, address to, uint amount );
    constructor() public {
        owner = msg.sender;
        balance = 0;
    }
    fallback() external  {
        emit PrintLog("Fallback called");
    }
    function forwardTo(address payable _receiver) public payable onlyOwner {
        _receiver.transfer(msg.value);
        emit Sent(msg.sender, _receiver, msg.value);
    }
    function getBalance() public view returns(uint, uint) {
        return (balance, address(this).balance);
    }
    function deposit(uint amount) public payable onlyAfter {
        timeToDeposit = block.timestamp + 10 seconds;
        require(msg.value == amount);
        balance += amount;
    }
    function withdrawAll() public onlyOwner minBalance {
        balance -= address(this).balance;
        payable(msg.sender).transfer(address(this).balance); 
    }
    modifier onlyOwner {
        require(msg.sender == owner);
        _;
    }
    modifier onlyAfter {
        require(block.timestamp >= timeToDeposit);
        _;
    }
    modifier minBalance {
        require(address(this).balance>101 wei);
        _;
    }
}

Writing src/BankV3.sol


## 2단계 컴파일

In [70]:
!solc --optimize --combined-json abi,bin src/BankV3.sol > src/BankV3.json

## 3단계 컨트랙 배포
이번에는 async, await로 변경하여 코드를 작성해보자. 배포함수를 async function deploy()로 선언한다. 그리고 그 안에 await web3.eth.getAccounts()로 계정을 구해서 이를 활용한다. 컨트랙을 생성하기 위해 블록체인에 전송하는 await send() 함수도 비동기적으로 처리하여, 주소를 출력

In [71]:
%%writefile src/BankV3DeployFromFile.js
var Web3 = require('web3');
var _abiBinJson = require('./BankV3.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); // reading ['src/BankV3.sol:BankV3']
console.log("- contract name: ", contractName);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(_abi);      //JSON parsing needed!!
_bin=_abiBinJson.contracts[contractName].bin;

//console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 259210}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Writing src/BankV3DeployFromFile.js


In [72]:
!node src/BankV3DeployFromFile.js

- contract name:  [ 'src/BankV3.sol:BankV3' ]
Deploying the contract from 0xfcd386EABDd83dD20c6845BF9d3FA1d26AFf8454
hash: 0x19fa4752cbeb40298b81e936eb1dff4d1890ace7eadf22e5a1a5a2641d6c5373
---> The contract deployed to: 0x5Cf87e5EF217eF60C5bf3175DBb0848CC94829c7


## 4단계 사용
10초 이내 저축

잔고 101보다 적은데 출금
```
블록체인에 send()가 필요한 함수는 비동기적으로 처리하기 위해 await로 처리한다. 비동기적으로 처리하면, 예를 들어 입금 deposit()하고 getBalance()하면 잔고에 입금분만큼 반영이 되어있는 것을 알 수 있다.
```

In [77]:
%%writefile src/BankV3Use.js
var Web3=require('web3');
var _abiBinJson = require('./BankV3.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/BankV3.sol:BankV3']
_abi=_abiBinJson.contracts[contractName].abi; //use just as read from file
_abiArray=JSON.parse(_abi);      //JSON parsing needed!!

console.log("- ABI: " + _abiArray);

var bank = new web3.eth.Contract(_abiArray,"0x5Cf87e5EF217eF60C5bf3175DBb0848CC94829c7");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    bank.methods.getBalance().call().then(console.log);
    await bank.methods.deposit(111).send({from: accounts[0], value:111});
    bank.methods.getBalance().call().then(console.log);
    await bank.methods.withdrawAll().send({from: accounts[0]});    //greater than 101
    bank.methods.getBalance().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));    
}
doIt()

Overwriting src/BankV3Use.js


In [78]:
!node src/BankV3Use.js

- ABI: [object Object],[object Object],[object Object],[object Object],[object Object],[object Object],[object Object],[object Object]
Account: 0xfcd386EABDd83dD20c6845BF9d3FA1d26AFf8454
Balance before: 999997930375999986113
Result { '0': '0', '1': '0' }
Result { '0': '111', '1': '111' }
Balance after: 999997772193999986113
Balance diff: 158182000033792
Result { '0': '0', '1': '0' }


# -------------------------------------------------------------------------

# 이벤트
* 프로그램에 의해 감지될 수 있는 동작을 의미한다. 키보드에서 어떤 키를 누르거나, 마우스로 클릭하거나 하는 것이 모두 이벤트에 해당
* 블록체인에서의 이벤트는 이벤트는 로그에 기록이 된다. 그 로그를 계속 듣고 있다가, 자신이 원하는 것이 포착되면 그 것이 이벤트로 인식

## 단계 1. 이벤트 설정
event PrintLog()
## 단계 2. 함수를 호출할 때 이벤트가 발생하도록 연결한다.
함수를 호출되고 emit 명령어를 만나게 되면 이벤트가 발생하는데 이를 binding이라고 한다.
```
funcion fireEvent(){
    emit PrintLog() //이 시점에 이벤트가 발생한다. 즉, 로그에 PrintLog()가 적힌다.
}
```
## 단계 3. 이벤트 발생을 callback 함수로 리스닝한다.
events.PrintLog() 이런식으로 events.이벤트명으로 접근한다.
1) 이벤트를 생성하고 리스닝한다.
```
> var myEvent = myInstance.events.PrintLog({from: web3.eth.accounts[0]}, {
    fromBlock: 0,     //시작 블록 수
    toBlock: 'latest' //끝 블록 수
  });
> myEvent.watch(function (error, result) {
    if (!error) {
        console.log("Event triggered ===> ",result);
        process.exit(1);
    }
});
```

2) 또는 한 명령어로 합쳐서 다음과 같이 리스닝할 수 있다.
```
var myEvent = myInstance.events.PrintLog({fromBlock: 0}, fromfunction(error, event) {
    if (!error)
        console.log(event);
});
```
대부분 (2)를 사용한다.
## 단계 4. 함수호출로 이벤트 발생.
> myInstance.fireEvent();
## 단계 5. 이벤트 watch 중기
> myEvent.stopWatching();

### event indexing
### event overloading

# -------------------------------------------------------------------------

## 웹소켓 연결과 해제


In [79]:
%%writefile src/webSocketTest.js
var Web3 = require('web3');
const myProvider = new Web3.providers.WebsocketProvider("ws://localhost:8345", {
    clientConfig: {
        keepalive:true, keepaliveInterval:10000
    } 
  });
var web3 = new Web3(myProvider);
console.log("(1) websocket url: ", myProvider.connection.url); //web3.currentProvider.connection.url
myProvider.on('connect', function() {
    console.log("(2) connecting websocket: "+web3.currentProvider.connected);
    //myProvider.disconnect();
    web3.currentProvider.connection.close();
    console.log("(3) disconnecting Websocket: "+web3.currentProvider.connected);
});
myProvider.on('close', function() { console.log("--> Websocket closed"); });
myProvider.on('end', function() { console.log("--> Websocket ended"); });
myProvider.on('error', function(error) { console.error(error); });

Overwriting src/webSocketTest.js


In [80]:
!node src/webSocketTest.js

(1) websocket url:  ws://localhost:8345
(2) connecting websocket: true
(3) disconnecting Websocket: false
--> Websocket ended


# -------------------------------------------------------------------------

## 간단한 이벤트 발생
이벤트를 만들고 함수가 호출되는 시점에 발생하도록 해보자. 이벤트는 발생하는 시점에 "Hello World!" 문자열이 출력되도록 한다. 이벤트가 발생하는지는 클라이언트에서 리스닝해서 알아 낸다. 이벤트가 발생하면 로그에 기록이 되고, 리스닝한다는 것은 이런 로그의 기록에 이벤트가 발생했는지를 확인하는 것

## 1단계 컨트랙 개발

In [81]:
%%writefile src/EventTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract EventTest {
    event MyLog(string my);
    function myFunction() public {
        emit MyLog("Hello World!");
    }
}

Overwriting src/EventTest.sol


## 2단계 컴파일

In [82]:
!solc --optimize --combined-json abi,bin src/EventTest.sol > src/EventTest.json

## 3단계 컨트랙 배포

In [83]:
%%writefile src/EventTestDeployFromFile.js
var Web3 = require('web3');
var _abiBinJson = require('./EventTest.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); // reading ['src/EventTest.sol:EventTest']
console.log("- contract name: ", contractName);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(_abi);      //JSON parsing needed!!
_bin=_abiBinJson.contracts[contractName].bin;

//console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 259210}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/EventTestDeployFromFile.js


In [84]:
!node src/EventTestDeployFromFile.js

- contract name:  [ 'src/EventTest.sol:EventTest' ]
Deploying the contract from 0xfcd386EABDd83dD20c6845BF9d3FA1d26AFf8454
hash: 0x6646ba7b610d3d1750701f94bed76c62b9564cad7d3f26317ef919ccd27d1dd9
---> The contract deployed to: 0xaF210FA55a6AcB051fB6E927654a1E32cdAE7324


* 위에서 주어진 transactionHash를 가지고 처리결과를 알 수 있다. gas 사용량, contractAddress도 찾을 수 있다.

In [88]:
!geth --exec "eth.getTransactionReceipt('0x6646ba7b610d3d1750701f94bed76c62b9564cad7d3f26317ef919ccd27d1dd9');" attach http://localhost:8345

{
  blockHash: "0xd46d715907817c3c3c3f1b2574be9ebb4cf78545afba5d74d8dc36ff9e0b2076",
  blockNumber: 23,
  contractAddress: "0xaf210fa55a6acb051fb6e927654a1e32cdae7324",
  cumulativeGasUsed: 94043,
  effectiveGasPrice: "0x77359400",
  from: "0xfcd386eabdd83dd20c6845bf9d3fa1d26aff8454",
  gasUsed: 94043,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  status: "0x1",
  to: null,
  transactionHash: "0x6646ba7b610d3d1750701f94bed76c62b9564cad7d3f26317ef919ccd27d1dd9",
  transactionIndex: 0,
  type: "0x0

## 4단계 사용

In [93]:
%%writefile src/EventTestHttpNoEventFiredUse.js
var Web3=require('web3');
var _abiBinJson = require('./EventTest.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/EventTest.sol:EventTest']

_abi=_abiBinJson.contracts[contractName].abi;

_abiArray=JSON.parse(_abi);      //JSON parsing needed!!
//_bin=_abiBinJson.contracts[contractName].bin;
console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);
var _test = new web3.eth.Contract(_abiArray,"0xaF210FA55a6AcB051fB6E927654a1E32cdAE7324");
var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result.returnValues));
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    const value = await _test.methods.myFunction()
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));});
    console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}
doIt()

Overwriting src/EventTestHttpNoEventFiredUse.js


In [94]:
!node src/EventTestHttpNoEventFiredUse.js

- ABI: [object Object],[object Object]
Account: 0xfcd386EABDd83dD20c6845BF9d3FA1d26AFf8454
Balance before: 999997563043999986113
---> myFunction called {"0":"Hello World!","my":"Hello World!"}
Balance after: 999997540240999986113
Balance diff: 22803000066048


## WebSocketProvider
파일에 로그 쓰기
이벤트가 발생하면 다음 시나리오를 생각해보자. 로컬에 다른 프로세스를 호출하는 작업을 할 수 있다. 다른 함수, 예를 들면, 로컬에서 주문상품을 배송하게 할 수 있다. 파일에 쓰는 시나리오도 가능하다.

fs.writeFile(파일명, 데이터, 인코딩방식, callback함수)함수를 사용하여 이벤트 발생 로그를 로컬 파일에 써보자. fs.appendFile()은 파일에 추가하는 기능이다.

In [99]:
%%writefile src/EventTestWsUse.js
var Web3=require('web3');
var fs = require('fs');
var _abiBinJson = require('./EventTest.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/EventTest.sol:EventTest']

_abi=_abiBinJson.contracts[contractName].abi; //use just as read from file

_abiArray=JSON.parse(_abi);      //JSON parsing needed!!

console.log("- ABI: " + _abiArray);


async function doIt() {
    var _test = new web3.eth.Contract(_abiArray, '0xaF210FA55a6AcB051fB6E927654a1E32cdAE7324');
    var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
        if (!error) {
            log = JSON.stringify(result.returnValues);
            console.log("Event fired: " + log);
            //fs.writeFile("src/EventTestLog.txt", log, "utf-8", function(e) {
            fs.appendFile("src/EventTestLog.txt", log, "utf-8", function(e) {
                if(!e) {
                    console.log(">> Writing to file");
                }
            });
        }
    });
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    const value = await _test.methods.myFunction()
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));});
    console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1); //force exit to disconnect websocket
}

doIt()

Overwriting src/EventTestWsUse.js


In [100]:
!node src/EventTestWsUse.js

- ABI: [object Object],[object Object]
Account: 0xfcd386EABDd83dD20c6845BF9d3FA1d26AFf8454
Balance before: 999997498112999986113
Event fired: {"0":"Hello World!","my":"Hello World!"}
---> myFunction called {"0":"Hello World!","my":"Hello World!"}
>> Writing to file
Balance after: 999997475309999986113
Balance diff: 22802999934976


In [101]:
!type src\EventTestLog.txt

{"0":"Hello World!","my":"Hello World!"}{"0":"Hello World!","my":"Hello World!"}{"0":"Hello World!","my":"Hello World!"}


# -------------------------------------------------------------------------

# 실습문제: 주문하면서 복수의 이벤트 사용

## 1단계 컨트랙 개발

In [102]:
%%writefile src/OrderEvent.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract OrderEvent {
    uint unitPrice = 10;
    event OrderLog(string);
    event OrderLog(bytes2 _itemId, uint _value);
    event OrderLog(uint256 timestamp);
    event OrderLog(address indexed _from, bytes2 _itemId, uint indexed _value);

    function order(bytes2 _itemId, uint quantity) public payable {
        //uint256 orderTime = now;
        uint256 orderTime = block.timestamp;
        uint256 orderAmount = quantity * unitPrice;
        require(msg.value == orderAmount);
        emit OrderLog("Ordered");
        emit OrderLog(orderTime);
        emit OrderLog(msg.sender, _itemId, msg.value);
    }
}

Overwriting src/OrderEvent.sol


## 2단계 컴파일

In [103]:
!solc --optimize --combined-json abi,bin src/OrderEvent.sol > src/OrderEvent.json

## 3단계 컨트랙 배포
```
컴파일하고 출력되는 abi, bin을 복사해서 배포 프로그램에 넣어주자. 사설망에 배포하려면, 계정을 unlock하는 것을 잊지 말자.

> personal.unlockAccount(eth.accounts[0]);
Unlock account 0x21c704354d07f804bab01894e8b4eb4e0eba7451
Passphrase:
```

In [104]:
%%writefile src/OrderEventDeploy.js
var Web3 = require('web3');
var _abiBinJson = require('./OrderEvent.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); // reading ['src/OrderEvent.sol:OrderEvent']
console.log("- contract name: ", contractName);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(_abi);      //JSON parsing needed!! o
_bin=_abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 259210}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Writing src/OrderEventDeploy.js


In [105]:
!node src/OrderEventDeploy.js

- contract name:  [ 'src/OrderEvent.sol:OrderEvent' ]
Deploying the contract from 0xfcd386EABDd83dD20c6845BF9d3FA1d26AFf8454
hash: 0xc518112c9c47b56c3422293d6035c08282e4bfc361bf813acb6414c96861c494
---> The contract deployed to: 0x0130339B4Fd0817f6DDf0F2d6C271D134BA16b05


* 해쉬 넣어 확인

In [106]:

!geth --exec "eth.getTransactionReceipt('0xc518112c9c47b56c3422293d6035c08282e4bfc361bf813acb6414c96861c494')" attach http://localhost:8345

{
  blockHash: "0x060391a9c797ea7e193056a0814e250e2d8781a700d5ba645aec65946161910d",
  blockNumber: 29,
  contractAddress: "0x0130339b4fd0817f6ddf0f2d6c271d134ba16b05",
  cumulativeGasUsed: 151387,
  effectiveGasPrice: "0x77359400",
  from: "0xfcd386eabdd83dd20c6845bf9d3fa1d26aff8454",
  gasUsed: 151387,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  status: "0x1",
  to: null,
  transactionHash: "0xc518112c9c47b56c3422293d6035c08282e4bfc361bf813acb6414c96861c494",
  transactionIndex: 0,
  type: "0

## 4단계 사용
```
event는 async방식으로 order()함수가 호출되면 실행된다. Order에 넘겨주는 매개변수는 바이트는 2바이트를 채워서 "0x1234", 주문량은 정수 3으로 맞추어 준다. 호출한 클라이언트에 event의 argument를 받아볼 수 있다 (예: result.args._from은 주소)

currentProvider는 MetaMask, Mist와 같은 Wallet을 사용하는 경우 자동으로 설정된다.
```

In [109]:
%%writefile src/OrderEventUse.js
var Web3=require('web3');
var _abiBinJson = require('./OrderEvent.json');      //importing a javascript file


var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/OrderEvent.sol:OrderEvent']

_abi=_abiBinJson.contracts[contractName].abi;

_abiArray=JSON.parse(_abi);      //JSON parsing needed!!
console.log("- ABI: " + _abiArray);


async function doIt() {
    var _order = new web3.eth.Contract(_abiArray, '0x0130339B4Fd0817f6DDf0F2d6C271D134BA16b05');
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    var event = _order.events.OrderLog({
            filter: {_from: accounts[0], _value: 30},
            fromBlock: 'latest', toBlock: 'pending'
        }, function (error, result) {
        if (!error) {
            console.log("Event fired: " + JSON.stringify(result.returnValues));
        }
    });
    var value;
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    // this will fire an event
    my = await _order.methods.order("0x1234", 3)
        .send({from: accounts[0], gas: 100000, value:30})
        //.then(function(my) {console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    // this will fire another event
    my = await _order.methods.order("0x1234", 4).send({from: accounts[0], gas: 100000, value:40});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    // this will NOT fire another eventㅣ
    my = await _order.methods.order("0x1234", 10).send({from: accounts[0], gas: 100000, value:100});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));    
    process.exit(1); //force exit to disconnect websocket
}

doIt()

Overwriting src/OrderEventUse.js


In [110]:
!node src/OrderEventUse.js

- ABI: [object Object],[object Object],[object Object],[object Object],[object Object]
Account: 0xfcd386EABDd83dD20c6845BF9d3FA1d26AFf8454
Balance before: 999997129823999986083
Event fired: {"0":"0xfcd386EABDd83dD20c6845BF9d3FA1d26AFf8454","1":"0x1234","2":"30","_from":"0xfcd386EABDd83dD20c6845BF9d3FA1d26AFf8454","_itemId":"0x1234","_value":"30"}
---> MyFunction called undefined
---> MyFunction called undefined
---> MyFunction called undefined
Balance after: 999996960719999985913
Balance diff: 169104000024576


# -------------------------------------------------------------------------

## 실습문제 - 주문하면서 복수의 이벤트 사용

## 1단계 컨트랙 개발

In [1]:
%%writefile src/OrderEvent.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract OrderEvent {
    uint unitPrice = 10;
    event OrderLog(string);
    event OrderLog(bytes2 _itemId, uint _value);
    event OrderLog(uint256 timestamp);
    event OrderLog(address indexed _from, bytes2 _itemId, uint indexed _value);

    function order(bytes2 _itemId, uint quantity) public payable {
        //uint256 orderTime = now;
        uint256 orderTime = block.timestamp;
        uint256 orderAmount = quantity * unitPrice;
        require(msg.value == orderAmount);
        emit OrderLog("Ordered");
        emit OrderLog(orderTime);
        emit OrderLog(msg.sender, _itemId, msg.value);
    }
}

Overwriting src/OrderEvent.sol


## 2단계 컴파일

In [2]:
!solc --optimize --combined-json abi,bin src/OrderEvent.sol > src/OrderEvent.json

## 3단계 컨트랙 배포

In [3]:
%%writefile src/OrderEventDeploy.js
var Web3 = require('web3');
var _abiBinJson = require('./OrderEvent.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); // reading ['src/OrderEvent.sol:OrderEvent']
console.log("- contract name: ", contractName);
_abi=_abiBinJson.contracts[contractName].abi;

_abiArray=JSON.parse(_abi);      //JSON parsing needed!!
_bin=_abiBinJson.contracts[contractName].bin;



async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 259210}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/OrderEventDeploy.js


In [4]:
!node src/OrderEventDeploy.js

- contract name:  [ 'src/OrderEvent.sol:OrderEvent' ]
Deploying the contract from 0x38B90A15f126760Ccd539D9A9fbb8c10bd37c02b
hash: 0xe16b08737c3dd20d4d9c761fb65e1c9bd48809abf7b39bad1c9e6d59b614d5d6
---> The contract deployed to: 0xd4d3b0b5d6C03C815187C6b75D1672063A1Dc95C


In [5]:
!geth --exec "eth.getTransactionReceipt('0xe16b08737c3dd20d4d9c761fb65e1c9bd48809abf7b39bad1c9e6d59b614d5d6')" attach http://localhost:8345

{
  blockHash: "0x256dc5951d2a435bb2a977d4801b47a3448ab5d1a5f3e09a040d4ca21ee27c68",
  blockNumber: 1,
  contractAddress: "0xd4d3b0b5d6c03c815187c6b75d1672063a1dc95c",
  cumulativeGasUsed: 151387,
  effectiveGasPrice: "0x77359400",
  from: "0x38b90a15f126760ccd539d9a9fbb8c10bd37c02b",
  gasUsed: 151387,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  status: "0x1",
  to: null,
  transactionHash: "0xe16b08737c3dd20d4d9c761fb65e1c9bd48809abf7b39bad1c9e6d59b614d5d6",
  transactionIndex: 0,
  type: "0x

## 4단계 사용
event는 async방식으로 order()함수가 호출되면 실행된다.

currentProvider는 MetaMask, Mist와 같은 Wallet을 사용하는 경우 자동으로 설정된다.

In [8]:
%%writefile src/OrderEventUse.js
var Web3=require('web3');
var _abiBinJson = require('./OrderEvent.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/OrderEvent.sol:OrderEvent']

_abi=_abiBinJson.contracts[contractName].abi; //use just as read from file
_abiArray=JSON.parse(_abi);      //JSON parsing needed!!

console.log("- ABI: " + _abiArray);

async function doIt() {
    var _order = new web3.eth.Contract(_abiArray, '0xd4d3b0b5d6C03C815187C6b75D1672063A1Dc95C');
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    var event = _order.events.OrderLog({
            filter: {_from: accounts[0], _value: 30},
            fromBlock: 'latest', toBlock: 'pending'
        }, function (error, result) {
        if (!error) {
            console.log("Event fired: " + JSON.stringify(result.returnValues));
        }
    });
    var value;
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    // this will fire an event
    my = await _order.methods.order("0x1234", 3)
        .send({from: accounts[0], gas: 100000, value:30})
        //.then(function(my) {console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    // this will fire another event
    my = await _order.methods.order("0x1234", 4).send({from: accounts[0], gas: 100000, value:40});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    // this will NOT fire another event
    my = await _order.methods.order("0x1234", 10).send({from: accounts[0], gas: 100000, value:100});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));    
    process.exit(1); //force exit to disconnect websocket
}

doIt()

Overwriting src/OrderEventUse.js


In [9]:
!node src/OrderEventUse.js

- ABI: [object Object],[object Object],[object Object],[object Object],[object Object]
Account: 0x38B90A15f126760Ccd539D9A9fbb8c10bd37c02b
Balance before: 999999654513999999970
Event fired: {"0":"0x38B90A15f126760Ccd539D9A9fbb8c10bd37c02b","1":"0x1234","2":"30","_from":"0x38B90A15f126760Ccd539D9A9fbb8c10bd37c02b","_itemId":"0x1234","_value":"30"}
---> MyFunction called undefined
---> MyFunction called undefined
---> MyFunction called undefined
Balance after: 999999485409999999800
Balance diff: 169104000024576


# -------------------------------------------------------------------------

# Fallback 함수
* 어떤 함수를 호출했는데, 그 함수가 없는 경우 fallback을 실행한다.
* 송금함수 호출시 없을 경우는 fallback() payable로 선언한다.
```
fallback() external payable {} //정상. 송금있는 경우 사용.
fallback() external {} //정상. 송금없는 경우 사용.
fallback() external payable { emit PrintLog("fallback");} //정상
fallback(bytes calldata _input) external returns(bytes memory _output) {} //정상. 그러나 최신버전에서만.
```

## 실습 - 존재하지 않는 함수를 호출해서 fallback함수를 실행

## 1단계 컨트랙 개발

In [10]:
%%writefile src/FallbackTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract FallbackTest {
    event PrintLog(string);
    function callA () pure public returns(string memory){
        return "doing callA";
    }
    fallback () external {
        emit PrintLog("fallback called");
    }
}

Overwriting src/FallbackTest.sol


## 2단계 컴파일

In [11]:
!solc --optimize --combined-json abi,bin src/FallbackTest.sol > src/FallbackTest.json

## 3단계 컨트랙 배포

In [14]:
%%writefile src/FallbackTestDeployFromFile.js
var Web3 = require('web3');
var _abiBinJson = require('./FallbackTest.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); // reading ['src/FallbackTest.sol:FallbackTest']
console.log("- contract name: ", contractName);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(_abi);      //JSON parsing needed!
_bin=_abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/FallbackTestDeployFromFile.js


In [15]:
!node src/FallbackTestDeployFromFile.js

- contract name:  [ 'src/FallbackTest.sol:FallbackTest' ]
Deploying the contract from 0x38B90A15f126760Ccd539D9A9fbb8c10bd37c02b
hash: 0xcc94b943b46266c605c81f7928f9080eeac6f3ff45be7dddf91f9870f1a078a6
---> The contract deployed to: 0xb73e3836e3A3b4833516a8ea297c9A9D0E05E420


## 4단계 사용

In [17]:
%%writefile src/FallbackTestUseFromFile.js
var Web3=require('web3');
var _abiBinJson = require('./FallbackTest.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/FallbackTest.sol:FallbackTest']
console.log("- contract name: ", contractName); //or console.log(contractName[0]);
_abi=_abiBinJson.contracts[contractName].abi; //use just as read from file
_abiArray=JSON.parse(_abi);      //JSON parsing needed!!
//_bin=_abiBinJson.contracts[contractName].bin;
console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);
//The above ABI from file does not work. So ABI was copied and pasted as below -> It worked!
_abiArray=[{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"","type":"string"}],"name":"PrintLog","type":"event"},{"stateMutability":"nonpayable","type":"fallback"},{"inputs":[],"name":"callA","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"pure","type":"function"}];
async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    var _instance = new web3.eth.Contract(_abiArray, "0xb73e3836e3A3b4833516a8ea297c9A9D0E05E420");
    var event = _instance.events.PrintLog({fromBlock: 0}, function (error, result) {
        if (!error) {
            console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
        }
    });

    _instance.methods.callA().call().then(function(res) { console.log(res); });  //null
    //call without calling any method
    //await _instance.methods.callB().send({from:accounts[0], to: "0x3991e87b71cBFf94aA0718F341d8Ad4bCF969f36"}); //fail
    //await _instance.methods.callA().send({from:accounts[0], data:"0x1234"});  //empty calldata to call fallback -> fail
    web3.eth.sendTransaction({from:accounts[0], to:"0x305F89e9b9C91B0b242874d77Ef675b0eBAD437C"}); //fallback called
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1); //force exit
}
doIt()

Overwriting src/FallbackTestUseFromFile.js


In [18]:
!node src/FallbackTestUseFromFile.js

- contract name:  [ 'src/FallbackTest.sol:FallbackTest' ]
- ABI: [object Object],[object Object],[object Object]
Account: 0x38B90A15f126760Ccd539D9A9fbb8c10bd37c02b
Balance before: 999999230479999999800
Balance after: 999999230479999999800
Balance diff: 0


# -------------------------------------------------------------------------